In [5]:
import numpy as np
import cv2 as cv
import time
from playsound import playsound

parpadeos=0
cuento=False

face_cascade = cv.CascadeClassifier('haarcascade_frontalface_alt.xml')
eye_cascade = cv.CascadeClassifier('haarcascade_eye.xml')
 
loop_1 = True   #variable que permite entrenar antes de comenzar. Además asegura que encuentre los dos ojos
salida = cv.VideoWriter('videoSalida.avi',cv.VideoWriter_fourcc(*'XVID'),20.0,(640,480)) 
cap = cv.VideoCapture(0)
ret,frame = cap.read() 
while(ret):
    salida.write(frame)
    ret, frame = cap.read()
    #Preproceso la imagen
    grayImage = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    grayImage = cv.equalizeHist(grayImage)
    grayImage = cv.bilateralFilter(grayImage,5,1,1)
    faces = face_cascade.detectMultiScale(grayImage, 1.1, 8 ,minSize=(200,200))
    if(len(faces)>0):
        for (x,y,w,h) in faces:
            frame = cv.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
 
            #Defino ROI para rostro
            roi_face = grayImage[y:y+h,x:x+w]
            roi_face_clr = frame[y:y+h,x:x+w]
            eyes = eye_cascade.detectMultiScale(roi_face,1.35,5,minSize=(50,50),maxSize=(100,100))
            if len(eyes)>2: #Si encuentra más de 2 ojos los descarto
                eyes=np.zeros((2,4))
                
             #En los ojos hallados les dibujamos rectángulos
            else:
                for (ex,ey,ew,eh) in eyes:
                    ex=ex+x
                    ey=ey+y
                    cv.rectangle(frame,(ex,ey),(ex+ew,ey+eh),(255,0,255),2)
                
            #Arbol de decisiones sobre cuantos ojos abiertos
            if(len(eyes)==2):
                if(loop_1):
                    cv.putText(frame,"Presione c para continuar",(70,70),cv.FONT_HERSHEY_COMPLEX_SMALL, 1,(0,255,0),2)
                else:
                    cv.putText(frame,"Ojos abiertos detectados", (70,70),cv.FONT_HERSHEY_COMPLEX_SMALL, 1,(255,255,255),2)
                    cuento=False
            if(len(eyes)==1):
                if(loop_1):
                    cv.putText(frame,"Abra el otro ojo",(70,70),cv.FONT_HERSHEY_COMPLEX_SMALL, 1,(0,255,0),2)
                else:
                    cv.putText(frame,"Un ojo cerrado", (70,70),cv.FONT_HERSHEY_COMPLEX_SMALL, 1,(255,255,255),2)
                    cuento=False
            if(len(eyes)==0):
                
                if(loop_1):
                    cv.putText(frame,"No se detectan ojos", (70,70),cv.FONT_HERSHEY_COMPLEX_SMALL, 1,(0,0,255),2)
                else:
                    #Activo alarma
                    if (cuento):
                        if time.time()-t_ini>2:
                            cv.putText(frame,"Chofer dormido", (70,70),cv.FONT_HERSHEY_COMPLEX_SMALL, 2,(0,0,255),2)
                            playsound('beep-06.wav')
                            parpadeos = parpadeos + 1
                            print("Se detectaron : ", parpadeos, "parpadeos")
                            cuento=False
                    else:
                        cuento=True
                        t_ini=time.time()
                        
             
    else:
        cv.putText(frame,"No hay cara",(100,100),cv.FONT_HERSHEY_COMPLEX_SMALL, 2,(0,255,0),2)
 
    #Control de teclado
    cv.imshow('Captura',frame)
    a = cv.waitKey(1)
    if(a==ord('s')):
        break
    elif(a==ord('c') and loop_1):
    #Inicio la detección
        loop_1 = False
 
cap.release()
salida.release()
cv.destroyAllWindows()

Se detectaron :  1 parpadeos
Se detectaron :  2 parpadeos
